In [1]:
from scrapy.crawler import CrawlerProcess
from src.spiders.PkSpider import PkSpider

process = CrawlerProcess(settings={
    "FEEDS": {
        "scrapped_pages/output_pk.csv": {"format": "csv"},
    },
})
process.crawl(PkSpider)
process.start()

2024-11-24 19:28:26 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-24 19:28:26 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-10-10.0.22631-SP0
2024-11-24 19:28:26 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-24 19:28:26 [py.warnings] WARNING: C:\Users\oneks\PycharmProjects\PAD_Project\.venv\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

Se

In [17]:
import pandas as pd

pk_df = pd.read_csv(r"scrapped_pages/output_pk.csv", delimiter=",")
pk_df.head()

,name,price_old,price_new
0,Kawa w kapsułkach do ekspresów Nespresso® Capr...,NaN,NaN
1,Kawa w kapsułkach do ekspresów Nespresso® Capr...,NaN,NaN
2,"Zestaw kawy ziarnistej Caprisette, 4 x 250 g","150,80","102,50"
3,"Zestaw ziaren kawy Caprisette, 4 x 250 g w świ...","160,99","80,50"
4,"Zestaw kaw ziarnistych Caprisette TOP, 3 x 250 g","145,97","87,58"


In [18]:
pk_df.dropna(inplace=True)
print(pk_df)

                                                  name price_old price_new
2         Zestaw kawy ziarnistej Caprisette, 4 x 250 g    150,80    102,50
3    Zestaw ziaren kawy Caprisette, 4 x 250 g w świ...    160,99     80,50
4     Zestaw kaw ziarnistych Caprisette TOP, 3 x 250 g    145,97     87,58
5           Zestaw kawy mielonej Caprisette, 4 x 250 g    150,80    102,50
6    Zestaw kawy mielonej Caprisette, 4 x 250 g w ś...    160,99     80,50
..                                                 ...       ...       ...
314  Kapsułki do kawy Tassimo Latte Macchiato Caram...     25,99     24,69
315  Kapsułki do kawy Tassimo Morning Cafe XL (komp...     25,99     24,69
316  Kapsułki do kawy Tassimo L’OR Espresso Delizio...     25,99     24,69
317  Kapsułki do kawy Tassimo Caffè Crema Classico ...     25,99     24,69
323  Rokit Energy Uplift kawa w kapsułkach do Nespr...     27,79     22,23

[180 rows x 3 columns]


In [19]:
import re

def extract_quantity_and_weight(product_name):
    quantity_match = re.search(r'(\d+)\s*x', product_name)
    quantity = int(quantity_match.group(1)) if quantity_match else 1

    weight_match = re.search(r'(\d+)\s*(g|kg|szt)', product_name, re.IGNORECASE)
    if weight_match:
        weight_value = float(weight_match.group(1))
        weight_unit = weight_match.group(2).lower()
        if weight_unit == 'g':
            weight_value /= 1000
        elif weight_unit in ['kg', 'g','szt']:
            pass
    else:
        weight_value = None

    clean_name = re.sub(r'(\d+)\s*x\s*\d+\s*(g|kg|szt)', '', product_name, flags=re.IGNORECASE).strip()
    return quantity, weight_value, clean_name

pk_df['product_quantity'], pk_df['product_weight'], pk_df['name'] = zip(*pk_df['name'].apply(extract_quantity_and_weight))

pk_df.rename(columns={
    "name": "product_name",
    "price_old": "price_old [PLN]",
    "price_new": "new_price [PLN]",
}, inplace=True)

pk_df["new_price [PLN]"] = pk_df["new_price [PLN]"].str.replace(",", ".").astype(float)
pk_df["price_old [PLN]"] = pk_df["price_old [PLN]"].str.replace(",", ".").astype(float)
pk_df.head()

,product_name,price_old [PLN],new_price [PLN],product_quantity,product_weight
2,"Zestaw kawy ziarnistej Caprisette,",150.80,102.50,4,0.25
3,"Zestaw ziaren kawy Caprisette, w świątecznym ...",160.99,80.50,4,0.25
4,"Zestaw kaw ziarnistych Caprisette TOP,",145.97,87.58,3,0.25
5,"Zestaw kawy mielonej Caprisette,",150.80,102.50,4,0.25
6,"Zestaw kawy mielonej Caprisette, w świąteczny...",160.99,80.50,4,0.25


In [20]:
pk_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 2 to 323
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_name      180 non-null    object 
 1   price_old [PLN]   180 non-null    float64
 2   new_price [PLN]   180 non-null    float64
 3   product_quantity  180 non-null    int64  
 4   product_weight    168 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 8.4+ KB


In [21]:
missing_vals = pk_df[pk_df["product_weight"].isna()]
missing_vals.head()

,product_name,price_old [PLN],new_price [PLN],product_quantity,product_weight
24,Zestaw kaw ziarnistych Expert-Top,369.99,239.99,1,NaN
44,Zestaw kawy ziarnistej Kivu + Magnifico,284.98,213.74,1,NaN
47,Zestaw kawy ziarnistej Specialty Yirgacheffe +...,174.97,131.23,1,NaN
97,Zestaw kawy ziarnistej Parallel 12 + Parallel ...,449.97,269.98,1,NaN
132,Kawowy kalendarz adwentowy Przyjaciół Kawy Pre...,339.99,237.99,1,NaN


In [22]:
pk_df.to_csv(r"cleaned_data/pk_cleaned.csv", index=False, encoding='utf-8')